In [ ]:
MARKET = 'SNP'

In [2]:
import sys
import pathlib
import numpy as np
import pandas as pd
import yaml
import asyncio

from ib_insync import IB, util, Option, MarketOrder, Contract
from typing import Callable, Coroutine, Union

In [3]:
# Specific to Jupyter. Will be ignored in IDE / command-lines
import IPython as ipy
if ipy.get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
    import nest_asyncio
    nest_asyncio.apply()
    util.startLoop()
    pd.options.display.max_columns = None
    
    THIS_FOLDER = '' # Dummy for jupyter notebook's current folder

In [4]:
# Get capability to import programs from `asyncib` folder
cwd = pathlib.Path.cwd() # working directory from where python was initiated
DATAPATH = cwd.joinpath('data', MARKET.lower()) # path to store data files
LOGFILE = DATAPATH.joinpath('temp.log') # path to store log files

IBPATH = cwd.parent.parent.joinpath('asyncib') # where ib programs are stored

# append IBPATH to import programs.
if str(IBPATH) not in sys.path:  # Convert it to string!
    sys.path.append(str(IBPATH))
    
IBDATAPATH = IBPATH.joinpath('data', MARKET.lower())

In [5]:
# Get the host, port, cid
from engine import Vars

ibp = Vars(MARKET.upper())  # IB Parameters from var.yml
HOST, PORT, CID = ibp.HOST, ibp.PORT, ibp.CID

In [6]:
# Get the pickle files
from os import listdir
fs = listdir(DATAPATH)

files = [f for f in fs if f[-4:] == '.pkl']
for f in files:
    exec(f"{f.split('.')[0]} = pd.read_pickle(DATAPATH.joinpath(f))")
files

['df_chains.pkl',
 'df_ohlcs.pkl',
 'df_opts.pkl',
 'df_opt_margins.pkl',
 'df_opt_prices.pkl',
 'df_symlots.pkl',
 'df_unds.pkl',
 'df_und_margins.pkl',
 'df_und_prices.pkl',
 'qopts.pkl']

# Enhancing PnL

In [11]:
%%time
with IB().connect(ibp.HOST, ibp.PORT, ibp.CID) as ib:
    acct = ib.managedAccounts()[0]
    pnlobj = ib.reqPnL(acct)

    df_mgns = util.df(ib.accountValues())

Wall time: 183 ms


In [8]:
df_mgns.tag.unique()

array(['AccountCode', 'AccountOrGroup', 'AccountReady', 'AccountType',
       'AccruedCash', 'AccruedCash-C', 'AccruedCash-S', 'AccruedDividend',
       'AccruedDividend-C', 'AccruedDividend-S', 'AvailableFunds',
       'AvailableFunds-C', 'AvailableFunds-S', 'Billable', 'Billable-C',
       'Billable-S', 'BuyingPower', 'CashBalance', 'CorporateBondValue',
       'Currency', 'Cushion', 'DayTradesRemaining',
       'DayTradesRemainingT+1', 'DayTradesRemainingT+2',
       'DayTradesRemainingT+3', 'DayTradesRemainingT+4',
       'EquityWithLoanValue', 'EquityWithLoanValue-C',
       'EquityWithLoanValue-S', 'ExcessLiquidity', 'ExcessLiquidity-C',
       'ExcessLiquidity-S', 'ExchangeRate', 'FullAvailableFunds',
       'FullAvailableFunds-C', 'FullAvailableFunds-S',
       'FullExcessLiquidity', 'FullExcessLiquidity-C',
       'FullExcessLiquidity-S', 'FullInitMarginReq',
       'FullInitMarginReq-C', 'FullInitMarginReq-S', 'FullMaintMarginReq',
       'FullMaintMarginReq-C', 'FullMaintMar

In [9]:
df_mgns[df_mgns.tag.isin(['TotalCashValue', 'TotalCashBalance', 'ExchangeRate'])]

,account,tag,value,currency,modelCode
42,U8898867,ExchangeRate,1.00,BASE,
43,U8898867,ExchangeRate,0.7425696,SGD,
44,U8898867,ExchangeRate,1.00,USD,
148,U8898867,TotalCashBalance,425872.6486,BASE,
149,U8898867,TotalCashBalance,275884.74,SGD,
150,U8898867,TotalCashBalance,221223.29,USD,
151,U8898867,TotalCashValue,425872.65,USD,


In [10]:
margins = df_mgns.set_index('tag').value.to_dict()

In [ ]:
pd.to_numeric(pd.Series(margins), errors='ignore')